# Limpieza

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../src"))
from datos.limpieza import Limpieza
from api.obtener_clima_cr import obtener_clima_cr
from basedatos.conexion_sql import conexion_SQL


meteo = pd.DataFrame(obtener_clima_cr()["hourly"])
meteo['time'] = pd.to_datetime(meteo['time'])
meteo.head(2)

,time,cloud_cover,precipitation,apparent_temperature,wind_speed_10m,sunshine_duration
0,2023-01-01 00:00:00,15,0.0,27.7,21.4,0.0
1,2023-01-01 01:00:00,99,0.0,27.8,20.9,0.0


In [2]:
demanda_2023 = pd.read_csv("../data/raw/DemandaMW-2023.csv")
demanda_2024 = pd.read_csv("../data/raw/DemandaMW-2024.csv")

demanda_MW = pd.concat([demanda_2023, demanda_2024], axis=0, ignore_index=True)

demanda_MW['fechaHora'] = pd.to_datetime(demanda_MW["fechaHora"])
demanda_MW.head()

,fechaHora,MW,MW_P
0,2023-01-01 00:00:00,1023.34,988.09
1,2023-01-01 01:00:00,994.38,940.10
2,2023-01-01 02:00:00,945.35,903.21
3,2023-01-01 03:00:00,909.23,874.09
4,2023-01-01 04:00:00,887.49,869.97


In [3]:
MW_clima = pd.merge(demanda_MW.drop(columns='MW_P'), meteo, left_on='fechaHora', right_on='time', how='left')

limpiar = Limpieza(MW_clima)
limpiar.transformar_fecha('fechaHora')
limpiar.eliminar_cols(['fechaHora', 'time'])

In [4]:
limpiar.df.head(2)

,MW,cloud_cover,precipitation,apparent_temperature,wind_speed_10m,sunshine_duration,mes,dia_mes,hora,dia_sem,anio
0,1023.34,15,0.0,27.7,21.4,0.0,1,1,0,6,2023
1,994.38,99,0.0,27.8,20.9,0.0,1,1,1,6,2023


In [5]:
limpiar.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MW                    17544 non-null  float64
 1   cloud_cover           17544 non-null  int64  
 2   precipitation         17544 non-null  float64
 3   apparent_temperature  17544 non-null  float64
 4   wind_speed_10m        17544 non-null  float64
 5   sunshine_duration     17544 non-null  float64
 6   mes                   17544 non-null  int32  
 7   dia_mes               17544 non-null  int32  
 8   hora                  17544 non-null  int32  
 9   dia_sem               17544 non-null  int32  
 10  anio                  17544 non-null  int32  
dtypes: float64(5), int32(5), int64(1)
memory usage: 1.1 MB


In [6]:
limpiar.df.describe()

,MW,cloud_cover,precipitation,apparent_temperature,wind_speed_10m,sunshine_duration,mes,dia_mes,hora,dia_sem,anio
count,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000
mean,1437.083018,71.728682,0.202251,31.364888,23.110209,1460.042674,6.519836,15.738714,11.500000,2.997264,2023.500684
std,249.359659,35.902265,0.753987,2.144152,9.284638,1733.646890,3.449649,8.804172,6.922384,2.003472,0.500014
min,833.240000,0.000000,0.000000,23.300000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2023.000000
25%,1200.692500,38.000000,0.000000,29.900000,16.300000,0.000000,4.000000,8.000000,5.750000,1.000000,2023.000000
50%,1479.865000,96.000000,0.000000,31.400000,23.100000,0.000000,7.000000,16.000000,11.500000,3.000000,2024.000000
75%,1656.202500,100.000000,0.100000,32.800000,29.800000,3600.000000,10.000000,23.000000,17.250000,5.000000,2024.000000
max,1922.780000,100.000000,20.300000,39.700000,57.100000,3600.000000,12.000000,31.000000,23.000000,6.000000,2024.000000


# Almacenamiento

In [ ]:
db = conexion_SQL("DESKTOP-V5VAI95\\SQLEXPRESS")

db.df_a_sql(limpiar.df, "Consumo")

In [ ]:
db.consultar("SELECT TOP 5 * FROM Consumo")

,MW,cloud_cover,precipitation,apparent_temperature,wind_speed_10m,sunshine_duration,mes,dia_mes,hora,dia_sem,anio
0,1023.34,15,0.0,27.7,21.4,0.00,1,1,0,6,2023
1,994.38,99,0.0,27.8,20.9,0.00,1,1,1,6,2023
2,945.35,65,0.0,27.8,22.1,3457.26,1,1,2,6,2023
3,909.23,19,0.0,27.7,23.2,3600.00,1,1,3,6,2023
4,887.49,41,0.0,27.9,23.3,3600.00,1,1,4,6,2023
